# Data Preprocessing

In [1]:
import nltk
nltk.download("punkt")
nltk.download("wordnet")
nltk.download("semcor") 
nltk.download("stopwords")
nltk.download('omw-1.4')

from nltk import word_tokenize
from nltk.corpus import semcor 
from nltk.corpus import stopwords
from nltk.corpus import wordnet as wn
from nltk.stem import WordNetLemmatizer

import warnings
import numpy as np
from tqdm.notebook import tqdm
from string import punctuation
from num2words import num2words
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.metrics.pairwise import cosine_similarity
warnings.filterwarnings("ignore")

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\cheta\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\cheta\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package semcor to
[nltk_data]     C:\Users\cheta\AppData\Roaming\nltk_data...
[nltk_data]   Package semcor is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\cheta\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\cheta\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [2]:
semcor_tagged_sents = semcor.tagged_sents(tag='sem')
semcor_untagged_sents = semcor.sents()

In [3]:
import gensim.downloader as api

#online loading from api
from gensim.models import KeyedVectors
path = "./GoogleNews-vectors-negative300.bin"
word2vec = KeyedVectors.load_word2vec_format(path, binary=True)

#local loading
# word2vec = api.load('word2vec-google-news-300')

In [4]:
print(semcor_tagged_sents)
print(semcor_tagged_sents[1])
#word_net.synsets('dog')
 

[[['The'], Tree(Lemma('group.n.01.group'), [Tree('NE', ['Fulton', 'County', 'Grand', 'Jury'])]), Tree(Lemma('state.v.01.say'), ['said']), Tree(Lemma('friday.n.01.Friday'), ['Friday']), ['an'], Tree(Lemma('probe.n.01.investigation'), ['investigation']), ['of'], Tree(Lemma('atlanta.n.01.Atlanta'), ['Atlanta']), ["'s"], Tree(Lemma('late.s.03.recent'), ['recent']), Tree(Lemma('primary.n.01.primary_election'), ['primary', 'election']), Tree(Lemma('produce.v.04.produce'), ['produced']), ['``'], ['no'], Tree(Lemma('evidence.n.01.evidence'), ['evidence']), ["''"], ['that'], ['any'], Tree(Lemma('abnormality.n.04.irregularity'), ['irregularities']), Tree(Lemma('happen.v.01.take_place'), ['took', 'place']), ['.']], [['The'], Tree(Lemma('jury.n.01.jury'), ['jury']), Tree(Lemma('far.r.02.far'), ['further']), Tree(Lemma('state.v.01.say'), ['said']), ['in'], Tree(Lemma('term.n.02.term'), ['term']), Tree(Lemma('end.n.02.end'), ['end']), Tree(Lemma('presentment.n.01.presentment'), ['presentments']), ['

In [5]:
EXTRA_SW = [
    "''",
    "'s",
    "``"
]

SW = stopwords.words("english")
SW += [p for p in punctuation]
SW += EXTRA_SW
# if '(' in SW:
#     print(1)

In [6]:
vectors = {}
semcor_sents = semcor.sents()
# for sentence in semcor_sents:
#     for word in sentence:
#         if word in model_w2v:
#             vectors[word] = model_w2v.get_vector(word)
print(semcor_sents[0])

['The', 'Fulton', 'County', 'Grand', 'Jury', 'said', 'Friday', 'an', 'investigation', 'of', 'Atlanta', "'s", 'recent', 'primary', 'election', 'produced', '``', 'no', 'evidence', "''", 'that', 'any', 'irregularities', 'took', 'place', '.']


In [7]:
isinstance('Folton',str)

True

In [8]:
syn = wn.synsets('cricket')
 
print ("Synset name :  ", syn)
 
# Defining the word
print ("\nSynset meaning : ", syn[0].definition())
print(syn[0].hyponyms())
for w in syn:
    print(w.definition())
# print(syn[0].hyponyms()[1].definition())

Synset name :   [Synset('cricket.n.01'), Synset('cricket.n.02'), Synset('cricket.v.01')]

Synset meaning :  leaping insect; male makes chirping noises by rubbing the forewings together
[Synset('european_house_cricket.n.01'), Synset('field_cricket.n.01'), Synset('mole_cricket.n.01'), Synset('tree_cricket.n.01')]
leaping insect; male makes chirping noises by rubbing the forewings together
a game played with a ball and bat by two teams of 11 players; teams take turns trying to score runs
play cricket


In [9]:
semcor_tagged_sents

[[['The'], Tree(Lemma('group.n.01.group'), [Tree('NE', ['Fulton', 'County', 'Grand', 'Jury'])]), Tree(Lemma('state.v.01.say'), ['said']), Tree(Lemma('friday.n.01.Friday'), ['Friday']), ['an'], Tree(Lemma('probe.n.01.investigation'), ['investigation']), ['of'], Tree(Lemma('atlanta.n.01.Atlanta'), ['Atlanta']), ["'s"], Tree(Lemma('late.s.03.recent'), ['recent']), Tree(Lemma('primary.n.01.primary_election'), ['primary', 'election']), Tree(Lemma('produce.v.04.produce'), ['produced']), ['``'], ['no'], Tree(Lemma('evidence.n.01.evidence'), ['evidence']), ["''"], ['that'], ['any'], Tree(Lemma('abnormality.n.04.irregularity'), ['irregularities']), Tree(Lemma('happen.v.01.take_place'), ['took', 'place']), ['.']], [['The'], Tree(Lemma('jury.n.01.jury'), ['jury']), Tree(Lemma('far.r.02.far'), ['further']), Tree(Lemma('state.v.01.say'), ['said']), ['in'], Tree(Lemma('term.n.02.term'), ['term']), Tree(Lemma('end.n.02.end'), ['end']), Tree(Lemma('presentment.n.01.presentment'), ['presentments']), ['

In [12]:
def sentence_vector(sent):
    sent_vec = np.zeros(300)
    for word in sent:
        if word in word2vec:
            sent_vec += word2vec.get_vector(word)
    return sent_vec        

In [13]:
def isNumber(s):
    try:
        float(s)
        return True
    except ValueError:
        return False

In [14]:
def number2word(word):
    if isNumber(word) and word != 'infinity' and word != 'nan':
        return num2words(word)
    else: 
        return word

In [16]:

def most_similar_sense(word,sentence,tag_return=False):
    updated_sentence = [w for w in sentence if (w not in SW) and (w.isalnum())]
    if word in updated_sentence:
        updated_sentence.remove(word)
    if(len(updated_sentence) == 0):
        return 'none'
    
    context_vec = np.zeros(300)
    
    context_vec = sentence_vector(updated_sentence)
    
    sense_vec_dict = {}
    
    all_senses = wn.synsets(word)
    
    if len(all_senses)==0:
        return 'none'
    
    for sense in all_senses:
        sense_vec_dict[sense] = []
        sense_sent = sense.definition().split()
        sense_sent = [w for w in sense_sent if (w not in SW) and (w.isalnum())]
        sense_vec = sentence_vector(sense_sent)
        sense_vec_dict[sense].append(sense_vec)
        
        hyponym_sent_list = sense.hyponyms()
        for hyponym_sent in hyponym_sent_list:
            hyponym_sent = hyponym_sent.definition().split()
            hyponym_vec = sentence_vector(hyponym_sent)
            sense_vec_dict[sense].append(hyponym_vec)
        for hypernym in sense.hypernyms():
            hypernym_sent = hypernym.definition().split()
            hypernym_sent = [w for w in hypernym_sent if (w not in SW) and (w.isalnum())]
            hypernym_vec = sentence_vector(hypernym_sent)
            sense_vec_dict[sense].append(hypernym_vec)
    
    if len(sense_vec_dict.keys())==0:
        return 'none'
    cos_sim_list = []
    for key, value in sense_vec_dict.items():
        cos_sim_list.append((key,word2vec.cosine_similarities(context_vec,value).mean()))
    cos_sim_list.sort(key=lambda x: x[1], reverse=True)
    if tag_return:
        return str(cos_sim_list[0][0])[8:-2],cos_sim_list[0][0]
    else:
        return str(cos_sim_list[0][0])[8:-2]

In [17]:

most_similar_sense('school',['The','boy','goes','to','school','2','or','infinite','times','a','day','.'])


'school.n.05'

In [18]:
def predict(tagged_sents,untagged_sents):
    actual = []
    pred = []
    count = 0
    for i in range(len(tagged_sents)):
        for j in range(len(tagged_sents[i])):
            
            word = ''
            if isinstance(tagged_sents[i][j],nltk.Tree):
                try:
                    if tagged_sents[i][j].height() == 3:
                        for tree in tagged_sents[i][j]:
                            if(tree.label() == 'NE'):
                                word = "".join(tree.leaves())
                                actual.append('NE')
                                
                    else:
                        word = "".join(tagged_sents[i][j].leaves())
                        sense = tagged_sents[i][j].label().synset().name()
                        actual.append(sense)
                except:
                    
                    if (tagged_sents[i][j].label()=='NE'):
                        word = "".join(tagged_sents[i][j].leaves())
                        actual.append('NE')
                    else:
                        word = "".join(tagged_sents[i][j].leaves())
                        actual.append(sense)
                
            else:
                actual.append('none')
                word = "".join(tagged_sents[i][j])
            pred.append(most_similar_sense(word,untagged_sents[i]))
            
        if count%500 == 0:
            print("Steps Completed ",count)
        count += 1
    return actual,pred

In [19]:
semcor_tagged_sents = semcor.tagged_sents(tag='sem')
semcor_untagged_sents = semcor.sents()


In [20]:

actual,pred = predict(semcor_tagged_sents,semcor_untagged_sents)

Steps Completed  0
Steps Completed  500
Steps Completed  1000
Steps Completed  1500
Steps Completed  2000
Steps Completed  2500
Steps Completed  3000
Steps Completed  3500
Steps Completed  4000
Steps Completed  4500
Steps Completed  5000
Steps Completed  5500
Steps Completed  6000
Steps Completed  6500
Steps Completed  7000
Steps Completed  7500
Steps Completed  8000
Steps Completed  8500
Steps Completed  9000
Steps Completed  9500
Steps Completed  10000
Steps Completed  10500
Steps Completed  11000
Steps Completed  11500
Steps Completed  12000
Steps Completed  12500
Steps Completed  13000
Steps Completed  13500
Steps Completed  14000
Steps Completed  14500
Steps Completed  15000
Steps Completed  15500
Steps Completed  16000
Steps Completed  16500
Steps Completed  17000
Steps Completed  17500
Steps Completed  18000
Steps Completed  18500
Steps Completed  19000
Steps Completed  19500
Steps Completed  20000
Steps Completed  20500
Steps Completed  21000
Steps Completed  21500
Steps Comple

In [21]:
len(semcor_tagged_sents)

37176

In [22]:
print(len(actual))
print(len(pred))


778587
778587


In [23]:
senses = []
for i in range(len(semcor_tagged_sents)):
    for j in range(len(semcor_tagged_sents[i])):
        if isinstance(semcor_tagged_sents[i][j],nltk.Tree):
            try:
                if semcor_tagged_sents[i][j].height() == 3:
                    for tree in semcor_tagged_sents[i][j]:
                        if(tree.label() == 'NE'):
                            senses.append('NE')
                            
                else:
                    sense = semcor_tagged_sents[i][j].label().synset().name()
                    senses.append(sense)
                    
            except:
                if (semcor_tagged_sents[i][j].label()=='NE'):
                    senses.append('NE')
                else:
                    sense = semcor_tagged_sents[i][j].label()
                    senses.append(sense)
        else:
            senses.append('none')
senses.append('notag')
senses = list(set(senses))
senses = {senses[i]:i for i in range(len(senses))}


In [24]:
y_true = [senses[i] for i in actual]
y_pred = [senses.get(i, senses['notag']) for i in pred]

In [26]:
import sklearn.metrics as sm
print(sm.accuracy_score(y_true, y_pred))
print(sm.f1_score(y_true, y_pred, average='weighted'))
print(sm.precision_score(y_true, y_pred,average='weighted'))
print(sm.recall_score(y_true, y_pred,average='weighted'))


0.4922108897271596
0.5743500313937702
0.7567236936581859
0.4922108897271596


In [27]:
def test_sentence(sentence):
    unc_sen = sentence
    sentence = sentence.split()
    sentence = [w for w in sentence if (w not in SW) and (w.isalnum())]
    print("==="*40)
    print("Sentence: ",unc_sen)
    print("==="*40)
    for word in sentence:
        sense = most_similar_sense(word,sentence,True)
        if sense == 'none':
            print("Word: ",word)
            print("Sense: ", sense)
        else:
            print("Word: ",word)
            print("Sense: ", sense[0])
            print("Definition of sense: ", sense[1].definition())
        print('---'*20)
    print("==="*40)
        

In [28]:
new_sentence = ' What is the interest on this loan ?'
test_sentence(new_sentence)

Sentence:   What is the interest on this loan ?
Word:  What
Sense:  none
------------------------------------------------------------
Word:  interest
Sense:  interest.n.04
Definition of sense:  a fixed charge for borrowing money; usually a percentage of the amount borrowed
------------------------------------------------------------
Word:  loan
Sense:  loan.n.01
Definition of sense:  the temporary provision of money (usually at interest)
------------------------------------------------------------
